<a href="https://colab.research.google.com/github/RANJITHMUTHUSAMY005/TripFare-Prediction/blob/main/streamlit_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.7 MB/s eta 0:00:00


In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

In [11]:
# Load your trained regression model
with open('/content/label_encoder.pkl', 'rb') as file:
    model = pickle.load(file)

In [12]:
with open('/content/ohe_encoder.pkl','rb') as file:
  ohe_encoder = pickle.load(file)

In [16]:
%%writefile tripfare.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
import joblib


st.markdown("""
<style>
.header {
    color: #1E90FF;
    font-size: 3.5rem;
    font-weight: 700;
    margin-top: -20px;
    margin-bottom: 20px;
    text-align: center;
    text-shadow: 2px 2px 4px #aaa;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}
</style> <h1 class="header">🚖 <span class="emoji">💰</span> Trip Fare Predictor <span class="emoji">📈</span></h1>""", unsafe_allow_html=True)


# 1. Inputs for numeric features:
trip_distance = st.number_input("Trip Distance (miles)", min_value=0.0, step=0.1)
trip_duration = st.number_input("Trip Duration (minutes)", min_value=0.0, step=1.0)

# 2. Categorical inputs:
ratecode_options = [1, 2, 3, 4]  # your RatecodeID categories
payment_type_options = [1, 3]     # your payment_type categories
am_pm_options = ['AM', 'PM']      # assuming am_pm is categorical too

ratecode_id = st.selectbox("RatecodeID", options=ratecode_options)
payment_type = st.selectbox("Payment Type", options=payment_type_options)
am_pm = st.selectbox("AM or PM", options=am_pm_options)

# 3. Prepare categorical input DataFrame to encode
cat_df = pd.DataFrame({
    'RatecodeID': [ratecode_id],
    'payment_type': [payment_type],
})

# Transform using OneHotEncoder
ohe_encoder = joblib.load("/content/ohe_encoder.pkl")

encoded_cat = ohe_encoder.transform(cat_df)
if hasattr(encoded_cat, "toarray"):
    encoded_cat = encoded_cat.toarray()

#  Get the one-hot column names generated by encoder
encoded_col_names = ohe_encoder.get_feature_names_out(['RatecodeID', 'payment_type'])


am_pm_encoded = 0 if am_pm == 'AM' else 1

# 7. Build final DataFrame for model input
numeric_df = pd.DataFrame({
    'trip_distance': [trip_distance],
    'trip_duration': [trip_duration],
    'am_pm': [am_pm_encoded]  # numeric encoded am_pm here
})

encoded_df = pd.DataFrame(encoded_cat, columns=encoded_col_names)

# Combine numeric + encoded categorical features
final_df = pd.concat([numeric_df, encoded_df], axis=1)

# 8. Reorder or add missing columns your model expects
expected_cols = ['trip_distance', 'trip_duration', 'am_pm',
                 'RatecodeID_1', 'RatecodeID_2', 'RatecodeID_4',
                 'payment_type_3', 'payment_type_1']

# Add missing expected columns if they don't exist (fill with 0)
for col in expected_cols:
    if col not in final_df.columns:
        final_df[col] = 0

# Select and reorder columns exactly as expected
final_df = final_df[expected_cols]

# 9. Predict button
if st.button("Predict Total Fare"):
    model= joblib.load("/content/best_model.pkl")
    pred = model.predict(final_df.values)
    st.write(f"Predicted Total Fare: ${pred[0]:.2f}")

Overwriting tripfare.py


In [5]:
!pip install -q streamlit streamlit_option_menu
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
import subprocess
subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"])
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 14.0 MB/s eta 0:00:00
--2026-02-25 08:12:45--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64 [following]
--2026-02-25 08:12:45--  https://github.com/cloudflare/cloudflared/releases/download/2026.2.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/f9298ca8-89c8-41fe-a51f-e24cb2059878?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-25T09%3A01%3A11Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-4

In [6]:
!streamlit run /content/tripfare.py &>/content/logs.txt &  # here instead of app.py please rename with your file name

In [7]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://express-payday-happened-tradition.trycloudflare.com
